https://www.kaggle.com/CVxTz/keras-bidirectional-lstm-baseline-lb-0-051

In [1]:
import gc
import numpy as np
import pandas as pd
import utils

from nltk.corpus import stopwords
from gensim.models import KeyedVectors
from tqdm import tqdm

from keras.preprocessing import text, sequence

/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: insp

Using TensorFlow backend.
/home/w/anaconda3/envs/idp3exp/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/w/anaconda3/envs/idp3exp/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/w/anaconda3/envs/idp3exp/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/w/anaconda3/envs/idp3exp/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/w/anaconda3/envs/idp3exp/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back 

/home/w/anaconda3/envs/idp3exp/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/w/anaconda3/envs/idp3exp/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/w/anaconda3/envs/idp3exp/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/w/anaconda3/envs/idp3exp/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:20: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is

In [2]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')

In [6]:
embedding_type = 'FastText2'
data_type = 'BasicClean2'
embedding_source = 'crawl-300d-2M.vec'

max_features = 200000
max_features_k = int(max_features / 1e3)
sequence_length = 320
embedding_dim = 300
create_embedding = True
random_init = False


emb_mean, emb_std = 0.020940498, 0.6441043
embedding_filename = '{}_{}_{}dim_{}k_{}len_random{}'.format(
    embedding_type, data_type, embedding_dim, max_features_k, sequence_length, int(random_init))
print(embedding_filename)


train, test = utils.load_data('../data/', data_type)

FastText2_BasicClean2_300dim_200k_160len_random0
Load data with basic cleaning with non-alphanumeric contained.


In [4]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
list_sentences_train = train["comment_text"].fillna("CVxTz").values
list_sentences_test = test["comment_text"].fillna("CVxTz").values

tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(train.comment_text.tolist() + test.comment_text.tolist())
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index)) + 1
print('Num of words: {}'.format(nb_words))

X_train = sequence.pad_sequences(list_tokenized_train, maxlen=sequence_length)
y_train = train[list_classes].values
X_test = sequence.pad_sequences(list_tokenized_test, maxlen=sequence_length)

del train, test, list_tokenized_train, list_tokenized_test
gc.collect()

Num of words: 200001


40

### FastText & GloVe.gensim

In [ ]:
if create_embedding:
    embedding_file = '/home/w/Projects/Toxic/data/embeddings/{}'.format(embedding_source)
    word2vec = KeyedVectors.load_word2vec_format(embedding_file, binary=False)
    print('Found %s word vectors of word2vec' % len(word2vec.vocab))
    if random_init:
        embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embedding_dim))
    else:
        embedding_matrix = np.zeros((nb_words, embedding_dim))
    for word, i in tqdm(word_index.items()):
        if word in word2vec.vocab:
            embedding_matrix[i] = word2vec.word_vec(word)
    print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
    pd.to_pickle(embedding_matrix, '../data/embeddings/{}.pkl'.format(embedding_filename))
else:
    embedding_matrix = pd.read_pickle('../data/embeddings/{}.pkl'.format(embedding_filename))

### GloVe